In [1]:
n = 100
win_size = 5

# using numpy

In [2]:
seq_np = np.random.randn(n)

In [3]:
from numpy.lib.stride_tricks import as_strided

def windows_np(seq_np):
    stride = seq_np.strides[0]
    return as_strided(
        seq_np, shape=(n - win_size + 1, win_size), strides=(stride, stride)
    )

def median_np():
    return np.median(windows_np(seq_np), axis=1)

In [4]:
%timeit median_np()

61.9 µs ± 1.02 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [5]:
result = list(median_np())

# using only standard library

In [6]:
from statistics import median

In [7]:
seq_list = list(seq_np)

### naive

In [8]:
def windows_naive(seq_list):
    for l, r in zip(range(n-win_size+1), range(win_size, n+1)):
        yield seq_list[l:r]

def median_naive():
    return [median(win) for win in windows_naive(seq_list)]

In [9]:
assert median_naive() == result

In [10]:
%timeit median_naive()

87.2 µs ± 753 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


### zip

In [11]:
def windows_zip(seq_list):
    return zip(*(seq_list[i:] for i in range(win_size)))

def median_zip():
    return [median(win) for win in windows_zip(seq_list)]

In [12]:
assert median_zip() == result

In [13]:
%timeit median_zip()

75.9 µs ± 431 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


### deque

In [14]:
from collections import deque

In [15]:
def windows_deque(seq_list):
    win = deque(seq_list[:win_size], maxlen=win_size)
    yield win
    for s in seq_list[win_size:]:
        win.append(s)
        yield win

def median_deque():
    return [median(win) for win in windows_deque(seq_list)]

In [16]:
assert median_deque() == result

In [17]:
%timeit median_deque()

86.1 µs ± 808 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


### zip + deque

In [18]:
def windows_zipdeq(seq_list):
    n_wins = n - win_size + 1
    
    def transposed_windows():
        t_win = deque(seq_list)
        for _ in range(win_size):
            yield t_win.copy()
            t_win.popleft()
        
    return zip(*transposed_windows())

def median_zipdeq():
    return [median(win) for win in windows_zipdeq(seq_list)]

In [19]:
assert median_zipdeq() == result

In [20]:
%timeit median_zipdeq()

89.2 µs ± 810 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


# result

numpy is the fastest.